## Step 1: Install Required Libraries
###🧩 What we're doing:
Installing SDKs for Semantic Kernel, FAISS, web APIs, and LLMs—so we can build skills, memory, and pipelines.

In [1]:
!pip install semantic-kernel faiss-cpu groq openai firecrawl-py newsapi-python python-dotenv semantic-kernel[faiss] sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.8/875.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.4/200.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
## Import Libraries and Load API Keys

### What Are We Doing?
# We’re bringing in the tools (libraries) we need and setting up API keys, like passwords to access Groq, OpenAI, Firecrawl, and News API. This is similar to connecting LangChain to APIs, but we’re using Semantic Kernel’s way. We also define a data model for FAISS to store policies.

import os
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion
from semantic_kernel.connectors.faiss import FaissStore, FaissCollection # Corrected import path based on documentation
from sentence_transformers import SentenceTransformer
from groq import Groq
from openai import OpenAI
from firecrawl import FirecrawlApp
from newsapi import NewsApiClient
import datetime
import json
import asyncio
import numpy as np
from pydantic import BaseModel
from google.colab import userdata

# Define data model for FAISS (required by SK)
class PolicyRecord(BaseModel):
    text: str
    embedding: list[float]

# Define a data model for FAISS (required by SK)
class PolicyRecord(BaseModel):
    text: str
    embedding: list

# Replace these with your actual API keys
GROQ_API_KEY = userdata.get('GROQ_API_KEY')  # Get from https://console.groq.com
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY4')  # Get from https://platform.openai.com
FIRECRAWL_API_KEY = userdata.get('FIRECRAWL_API_KEY')  # Get from https://www.firecrawl.dev
NEWSAPI_API_KEY = userdata.get('NEWS_API_KEY')  # Get from https://newsapi.org

# Verify API keys (basic check)
if not all([GROQ_API_KEY, OPENAI_API_KEY, FIRECRAWL_API_KEY, NEWSAPI_API_KEY]):
    raise ValueError("Please provide all API keys")

In [3]:
import json
import os
from datetime import datetime

# Create memory folder
memory_folder = "json_memory"
os.makedirs(memory_folder, exist_ok=True)

# Save memory to JSON
def save_to_json_memory(category, id, content):
    file_path = os.path.join(memory_folder, f"{category}.json")
    data = {}
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            data = json.load(f)
    data[id] = {
        "timestamp": datetime.now().isoformat(),
        "content": content
    }
    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

# Retrieve from JSON memory
def retrieve_from_json_memory(category, keyword):
    file_path = os.path.join(memory_folder, f"{category}.json")
    if not os.path.exists(file_path):
        return None
    with open(file_path, "r") as f:
        data = json.load(f)
    matches = [
        v["content"] for v in data.values()
        if keyword.lower() in v["content"].lower()
    ]
    return matches[-1] if matches else None

In [10]:
import requests
import openai
from openai import OpenAI # Import the new OpenAI client

openai.api_key = OPENAI_API_KEY  # This line might still be here for backward compatibility but the client is preferred

def news_search(query, api_key):
    url = f"https://newsapi.org/v2/everything?q={query}&language=en&pageSize=3&apiKey={api_key}"
    response = requests.get(url)
    data = response.json()
    return [article["content"] or article["description"] for article in data["articles"]]

def summarize_with_openai(content, api_key): # Pass api_key to the function
    client = OpenAI(api_key=api_key) # Initialize the client with the API key
    prompt = f"Summarize the following in simple language:\n\n{content}\n\nSummary:"
    response = client.chat.completions.create( # Use the new client syntax
        model="gpt-4o-mini",  # or "llama3" with Groq
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [11]:
# You can keep templates in text files or define inline
template = """
You are a formatter bot. Format the following news summaries clearly:

{{ $input }}

End with a closing statement.
"""

def format_report(content):
    return template.replace("{{ $input }}", content)

In [12]:
context = {}
context["query"] = "Artificial Intelligence"

In [14]:
# Setup
news_api_key = NEWSAPI_API_KEY
openai_api_key = OPENAI_API_KEY # Get the OpenAI API key

# Step 1: Search
query = "Artificial Intelligence"
articles = news_search(query, news_api_key)

# Step 2: Summarize each
summaries = [summarize_with_openai(article, openai_api_key) for article in articles] # Pass openai_api_key

# Step 3: Save to memory
for i, summary in enumerate(summaries):
    save_to_json_memory("summaries", f"{query}_{i}", summary)

# Step 4: Format report
final_report = format_report("\n\n".join(summaries))
print(final_report)


You are a formatter bot. Format the following news summaries clearly:

Microsoft and OpenAI are set to announce the opening of an AI training center for teachers in the second-largest teachers union in the US.

A small part of the contract between OpenAI and Microsoft, which used to seem unlikely, has now become a major issue in their important partnership.

Delta Air Lines plans to use artificial intelligence to help set 20% of its ticket prices by the end of 2025. This means they want to use technology to adjust prices based on demand and other factors.

End with a closing statement.

